In [ ]:
import glob
import json

# Use glob to find all cropped images
cropped_pattern = 'static/output/*/*/cropped.json'
cropped_files = glob.glob(cropped_pattern)

print(f"Found {len(cropped_files)} cropped images")

results = []  # Store results for each cropped image
# Iterate over each cropped image
for cropped_file in cropped_files:
    with open(cropped_file, 'r') as f:
        result = json.load(f)
    results.append(result)

In [ ]:
import cv2

original_image_file = 'static/imgs_sections/2-SMC-1-AL_72,99m_n=_2,5x_cesc.jpg'
original_image = cv2.imread(original_image_file)


In [ ]:
# Alternative solution that works with any matplotlib backend
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import json
import os

%matplotlib widget

def create_simple_interactive_viewer():
    """Create an interactive image viewer that works with standard backends"""
    
    # Prepare data for the target image
    target_image_key = "2-SMC-1-AL_72,99m_n=_2,5x_cesc.jpg"
    image_data = []
    
    for it, result in enumerate(results):
        cropped_file = result["cropped_file"]
        path = os.path.dirname(cropped_file)
        
        options_file = f'{path}/options.json'
        with open(options_file, 'r') as f:
            options = json.load(f)
        source_image_key = options["image_select.filename"]
        
        if source_image_key != target_image_key:
            continue
        
        # loading review text
        review_text = "N/A"
        if os.path.exists(f'{path}/end_review.txt'):
            try:
                with open(f'{path}/end_review.txt', 'r', encoding='utf-8') as f:
                    review_text = f.read().strip()
            except:
                with open(f'{path}/end_review.txt', 'r') as f:
                    review_text = f.read().strip()

        top_left = result['top_left']
        cropped_shape = result['cropped_shape']
        
        # Create rectangle bounds
        x, y = top_left
        width, height = cropped_shape[1], cropped_shape[0]
        
        image_data.append({
            'x': x,
            'y': y,
            'width': width,
            'height': height,
            'cropped_file': cropped_file,
            'options': options,
            'match_val': result['max_val'],
            'index': it,
            'review': review_text,
        })
    
    # Create the figure and axis
    fig, ax = plt.subplots(figsize=(15, 10))
    
    # Display the original image
    original_img = original_image[:, :, ::-1]  # Convert BGR to RGB
    ax.imshow(original_img)
    ax.set_title("Interactive Cropped Regions Viewer\n(Click on rectangles to see file info)")
    ax.axis('off')
    
    # Create rectangles and store them
    rectangles = []
    cmap = plt.get_cmap('rainbow', len(image_data))
    
    for i, data in enumerate(image_data):
        if data["options"].get("user", {}).get('experience', '') <= 2:
            continue
        if data.get('review', '') == "":
            continue
        # Create rectangle
        rgba_color = cmap(((i * 97) % len(image_data)) / len(image_data))
        rect = patches.Rectangle(
            (data['x'], data['y']), 
            data['width'],
            data['height'],
            linewidth=3,
            edgecolor=rgba_color[:3],  # RGB without alpha
            facecolor='none',
            alpha=0.7,
            picker=True  # Make rectangle clickable
        )
        ax.add_patch(rect)
        rectangles.append((rect, data))
    
    rectangles.sort(key=lambda x: x[1]['width']*x[1]['height'])  # Sort by size
    
    # Create text box for displaying information
    text_box = ax.text(0.02, 0.98, "Click on a colored rectangle to see file information", 
                      transform=ax.transAxes, 
                      fontsize=10, verticalalignment='top',
                      bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
    
    def on_click(event):
        """Handle mouse click events"""
        if event.inaxes != ax:
            return
            
        # Check if click is on any rectangle
        for rect, data in rectangles:
            if rect.contains(event)[0]:
                                
                # Create clipboard-friendly text
                clipboard_text = f"""File: {os.path.basename(data['cropped_file'])}
Full Path: {data['cropped_file']}
Match Quality: {data['match_val']:.4f}
Position: ({data['x']}, {data['y']})
Size: {data['width']} x {data['height']}
User Name: {data['options'].get("user", {}).get('name', 'N/A')}
User E-mail: {data['options'].get("user", {}).get('email', 'N/A')}
Experience Level: {data['options'].get("user", {}).get('experience', 'N/A')}
Review: {data.get('review', 'N/A')}
Index: {data['index']}"""
                
                # Copy to clipboard
                import pyperclip
                pyperclip.copy(clipboard_text)
                display_text = clipboard_text + "\n\n✅ Copied to clipboard!"

                text_box.set_text(display_text)
                fig.canvas.draw()
                return
    
    # Connect the click event
    fig.canvas.mpl_connect('button_press_event', on_click)
    
    # Add a legend explaining the interaction
    legend_text = "💡 Click on colored rectangles to see file information and copy to clipboard"
    ax.text(0.02, 0.02, legend_text, transform=ax.transAxes, 
           fontsize=12, verticalalignment='bottom',
           bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7))
    
    plt.tight_layout()
    plt.show()
    
    return fig, ax, image_data

# Create the simple interactive viewer
print("Creating click-based interactive viewer...")
fig_simple, ax_simple, data_simple = create_simple_interactive_viewer()